##    *LSTM_Zoom*


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

Saving Zoom.xlsb to Zoom.xlsb


In [ ]:
!pip install pyxlsb


In [ ]:
import pandas as pd
# Access the uploaded Excel file using its filename
file_name = next(iter(uploaded.keys()))
# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_name)

In [ ]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.024571,192.0.0.2,144.195.172.120,UDP,109,55252 > 8801 Len=81
1,2,0.024571,192.0.0.2,144.195.172.120,UDP,1090,47990 > 8801 Len=1062
2,3,0.024571,192.0.0.2,144.195.172.120,UDP,604,47990 > 8801 Len=576
3,4,0.024571,192.0.0.2,144.195.172.120,UDP,105,47990 > 8801 Len=77
4,6,0.024571,192.0.0.2,144.195.172.120,UDP,116,55252 > 8801 Len=88


In [ ]:
# Check for duplicates in the dataset
print("Number of duplicates:", df.duplicated().sum())

# Check for missing values in the dataset
print(df.isnull().sum())

# Handle missing values (if any)
# For simplicity, let's drop any rows with missing values
df.dropna(inplace=True)

Number of duplicates: 0
No.            0
Time           0
Source         0
Destination    0
Protocol       0
Length         0
Info           0
dtype: int64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Update this line
from sklearn.metrics import r2_score


In [ ]:
# Convert IP addresses to numerical values
ip_encoder = LabelEncoder()
df['Source'] = ip_encoder.fit_transform(df['Source'])
df['Destination'] = ip_encoder.transform(df['Destination'])


In [ ]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.024571,3,2,UDP,109,55252 > 8801 Len=81
1,2,0.024571,3,2,UDP,1090,47990 > 8801 Len=1062
2,3,0.024571,3,2,UDP,604,47990 > 8801 Len=576
3,4,0.024571,3,2,UDP,105,47990 > 8801 Len=77
4,6,0.024571,3,2,UDP,116,55252 > 8801 Len=88


In [ ]:
# Convert "Protocol" column to numerical values using LabelEncoder
protocol_encoder = LabelEncoder()
df['Protocol'] = protocol_encoder.fit_transform(df['Protocol'])

# Check the unique values in the "Protocol" column after encoding
print("Unique values in Protocol column after encoding:", df['Protocol'].unique())


Unique values in Protocol column after encoding: [2 3 1 0]


In [ ]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.024571,3,2,2,109,55252 > 8801 Len=81
1,2,0.024571,3,2,2,1090,47990 > 8801 Len=1062
2,3,0.024571,3,2,2,604,47990 > 8801 Len=576
3,4,0.024571,3,2,2,105,47990 > 8801 Len=77
4,6,0.024571,3,2,2,116,55252 > 8801 Len=88


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming you have loaded your dataset into 'df'

# Preprocess the 'Time' column to convert it into numerical values (seconds)
def time_to_seconds(time_str):
    if isinstance(time_str, str):
        time_components = time_str.split(':')
        minutes = int(time_components[0])
        seconds_and_milliseconds = time_components[1].split('.')
        seconds = int(seconds_and_milliseconds[0])
        milliseconds = int(seconds_and_milliseconds[1])
        return minutes * 60 + seconds + milliseconds / 1000.0
    else:
        return time_str

# Create a new column 'TimeSeconds' to store the time in seconds
df['TimeSeconds'] = df['Time'].apply(time_to_seconds)

# Now, you can proceed with the normalization using MinMaxScaler
time_scaler = MinMaxScaler()
df['TimeSeconds'] = time_scaler.fit_transform(df['TimeSeconds'].values.reshape(-1, 1))

# The 'TimeSeconds' column now contains normalized numerical values between 0 and 1
print(df['TimeSeconds'])

0          0.589711
1          0.589711
2          0.589711
3          0.589711
4          0.589711
             ...   
1048570    0.930804
1048571    0.930804
1048572    0.930804
1048573    0.930804
1048574    0.930804
Name: TimeSeconds, Length: 1048575, dtype: float64


In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert the 'Info' column to strings
df['Info'] = df['Info'].astype(str)

# Step 1: Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Info'])
df['Info_tokens'] = tokenizer.texts_to_sequences(df['Info'])

# Step 2: Padding
max_sequence_length = 10  # Set your desired sequence length
df['Info_padded'] = pad_sequences(df['Info_tokens'], maxlen=max_sequence_length, padding='post').tolist()


In [ ]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info,TimeSeconds,Info_tokens,Info_padded
0,1,0.024571,3,2,2,109,55252 > 8801 Len=81,0.589711,"[1251, 2, 1, 7]","[1251, 2, 1, 7, 0, 0, 0, 0, 0, 0]"
1,2,0.024571,3,2,2,1090,47990 > 8801 Len=1062,0.589711,"[1236, 2, 1, 658]","[1236, 2, 1, 658, 0, 0, 0, 0, 0, 0]"
2,3,0.024571,3,2,2,604,47990 > 8801 Len=576,0.589711,"[1236, 2, 1, 862]","[1236, 2, 1, 862, 0, 0, 0, 0, 0, 0]"
3,4,0.024571,3,2,2,105,47990 > 8801 Len=77,0.589711,"[1236, 2, 1, 1002]","[1236, 2, 1, 1002, 0, 0, 0, 0, 0, 0]"
4,6,0.024571,3,2,2,116,55252 > 8801 Len=88,0.589711,"[1251, 2, 1, 5]","[1251, 2, 1, 5, 0, 0, 0, 0, 0, 0]"


In [ ]:
# !pip install --upgrade keras
!pip install tensorflow


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Step 1: Convert text data to list of strings (if it's not already)
if not isinstance(df['Info'], list):
    df['Info'] = [str(df['Info'][i]) for i in range(len(df['Info']))]

# Step 2: Tokenization and Padding for text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Info'])
X_text_train_seq = tokenizer.texts_to_sequences(df['Info'])

sequence_length =10

X_text_train_padded = pad_sequences(X_text_train_seq, padding='post', maxlen=sequence_length)

# Prepare input and output data
X_numerical = df[['TimeSeconds', 'Source', 'Destination', 'Protocol']].values
X_text = X_text_train_padded
y = df['Length'].values

# Split the data into training and testing sets
X_numerical_train, X_numerical_test, X_text_train_padded, X_text_test_padded, y_train, y_test = train_test_split(
    X_numerical, X_text, y, test_size=0.2, random_state=42
)

# Step 7: Build the LSTM model for numerical features
input_text = Input(shape=(sequence_length,))
embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32)(input_text)
lstm_text = LSTM(128)(embedding)
dropout_text = Dropout(0.2)(lstm_text)

In [ ]:
# Output layer for text features
output_text = Dense(1, activation='sigmoid')(dropout_text)
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Reshape

# Step 8: Build the LSTM model for numerical features
input_numerical = Input(shape=(X_numerical.shape[1],))
reshaped_numerical = Reshape((1, X_numerical.shape[1]))(input_numerical)
lstm_numerical = LSTM(128)(reshaped_numerical)
dropout_numerical = Dropout(0.2)(lstm_numerical)

In [ ]:
# Output layer for numerical features
output_numerical = Dense(1, activation='sigmoid')(dropout_numerical)

# Step 9: Create the final model by concatenating both text and numerical features
concatenated = tf.keras.layers.Concatenate()([output_text, output_numerical])
output = Dense(1, activation='sigmoid')(concatenated)

model = Model(inputs=[input_text, input_numerical], outputs=output)

In [ ]:
# Step 10: Compile and train the model
from tensorflow.keras.optimizers import Adam

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))


In [ ]:
# Assuming you have already defined the model architecture for the numerical and text features

# Step 11: Train the model with both numerical and text features
history = model.fit(
    [X_text_train_padded, X_numerical_train],
    y_train,
    epochs=200,
    batch_size=64,
    validation_split=0.3
)
# Evaluate the model on the test data
loss = model.evaluate([X_text_test_padded, X_numerical_test], y_test, verbose=0)
print("Test Loss:", loss)

# Make predictions on the test data
y_pred = model.predict([X_text_test_padded, X_numerical_test])

# Calculate R2 score
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)
import matplotlib.pyplot as plt

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()